# Building a Lead Scoring Model with EvalML

In this demo, we will build an optimized lead scoring model using EvalML. To optimize the pipeline, we will set up an objective function to maximize the revenue generated with true positives while taking into account the cost of false positives. At the end of this demo, we also show you how introducing the right objective during the training is over 6x better than using a generic machine learning metric like AUC.

In [1]:
import evalml
from evalml import AutoMLSearch
from evalml.objectives import LeadScoring

## Configure LeadScoring 

To optimize the pipelines toward the specific business needs of this model, you can set your own assumptions for how much value is gained through true positives and the cost associated with false positives. These parameters are

* `true_positive` - dollar amount to be gained with a successful lead
* `false_positive` - dollar amount to be lost with an unsuccessful lead

Using these parameters, EvalML builds a pileline that will maximize the amount of revenue per lead generated.

In [2]:
lead_scoring_objective = LeadScoring(
    true_positives=1000,
    false_positives=-10
)

## Dataset

We will be utilizing a dataset detailing a customer's job, country, state, zip, online action, the dollar amount of that action and whether they were a successful lead.

In [3]:
from urllib.request import urlopen
import pandas as pd

customers_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/customers.csv')
interactions_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/interactions.csv')
leads_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/previous_leads.csv')
customers = pd.read_csv(customers_data)
interactions = pd.read_csv(interactions_data)
leads = pd.read_csv(leads_data)

X = customers.merge(interactions, on='customer_id').merge(leads, on='customer_id')
y = X['label']

X = X.drop(['customer_id', 'date_registered', 'birthday','phone', 'email',
        'owner', 'company', 'id', 'time_x',
        'session', 'referrer', 'time_y', 'label', 'country'], axis=1)

display(X.head())

,job,state,zip,action,amount
0,"Engineer, mining",NY,60091.0,page_view,NaN
1,"Psychologist, forensic",CA,NaN,purchase,135.23
2,"Psychologist, forensic",CA,NaN,page_view,NaN
3,Air cabin crew,NaN,60091.0,download,NaN
4,Air cabin crew,NaN,60091.0,page_view,NaN


## Search for best pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

EvalML natively supports one-hot encoding and imputation so the above `NaN` and categorical values will be taken care of.

In [4]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=0.2, random_state=0)

print(X.dtypes)

job        object
state      object
zip       float64
action     object
amount    float64
dtype: object


Because the lead scoring labels are binary, we will use `AutoMLSearch(problem_type='binary')`. When we call `.search()`, the search for the best pipeline will begin. 

In [5]:
automl = AutoMLSearch(problem_type='binary',
                      objective=lead_scoring_objective,
                      additional_objectives=['auc'],
                      max_pipelines=5,
                      optimize_thresholds=True)

automl.search(X_train, y_train)

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Lead Scoring. 
Greater score is better.

Searching up to 5 pipelines. 
Allowed model families: xgboost, random_forest, catboost, linear_model



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

(1/5) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 42.140
(2/5) CatBoost Classifier w/ Simple Imputer    Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 44.246
(3/5) XGBoost Classifier w/ Simple Imputer ... Elapsed:00:23
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 41.327
(4/5) Random Forest Classifier w/ Simple Im... Elapsed:00:24
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 40.830
(5/5) Logistic Regression Classifier w/ Sim... Elapsed:00:26
	Starting cross validation
	Finished cross validation - mean Lead Scoring: 40.866

Search finished after 00:29            
Best pipeline: CatBoost Classifier w/ Simple Imputer
Best pipeline Lead Scoring: 44.246141


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the lead scoring objective we defined

In [6]:
automl.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,1,CatBoost Classifier w/ Simple Imputer,44.246141,False,{'Simple Imputer': {'impute_strategy': 'most_f...
1,0,Mode Baseline Binary Classification Pipeline,42.140250,False,{'Baseline Classifier': {'strategy': 'random_w...
2,2,XGBoost Classifier w/ Simple Imputer + One Hot...,41.327240,False,{'Simple Imputer': {'impute_strategy': 'most_f...
3,4,Logistic Regression Classifier w/ Simple Imput...,40.866142,False,{'Simple Imputer': {'impute_strategy': 'most_f...
4,3,Random Forest Classifier w/ Simple Imputer + O...,40.830334,False,{'Simple Imputer': {'impute_strategy': 'most_f...


to select the best pipeline we can run

In [7]:
best_pipeline = automl.best_pipeline

### Describe pipeline

You can get more details about any pipeline. Including how it performed on other objective functions.

In [8]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

*****************************************
* CatBoost Classifier w/ Simple Imputer *
*****************************************

Problem Type: Binary Classification
Model Family: CatBoost

Pipeline Steps
1. Simple Imputer
	 * impute_strategy : most_frequent
	 * fill_value : None
2. CatBoost Classifier
	 * n_estimators : 1000
	 * eta : 0.03
	 * max_depth : 6
	 * bootstrap_type : None

Training
Training for Binary Classification problems.
Objective to optimize binary classification pipeline thresholds for: <evalml.objectives.lead_scoring.LeadScoring object at 0x153eb7d90>
Total training time (including CV): 22.7 seconds

Cross Validation
----------------
             Lead Scoring   AUC # Training # Testing
0                  45.297 0.925   2479.000  1550.000
1                  42.884 0.924   2479.000  1550.000
2                  44.558 0.930   2480.000  1549.000
mean               44.246 0.926          -         -
std                 1.236 0.003          -         -
coef of var         0.0

## Evaluate on hold out

Finally, we retrain the best pipeline on all of the training data and evaluate on the holdout

In [9]:
best_pipeline.fit(X_train, y_train)

<evalml.pipelines.utils.make_pipeline.<locals>.GeneratedPipeline at 0x15510df70>

Now, we can score the pipeline on the hold out data using both the lead scoring score and the AUC.

In [10]:
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", lead_scoring_objective])

OrderedDict([('AUC', 0.9427319431852523),
             ('Lead Scoring', 11.986242476354256)])

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimizing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the lead scoring objective to see how the best pipelines compare.

In [11]:
automl_auc = evalml.AutoMLSearch(problem_type='binary',
                                 objective='auc',
                                 additional_objectives=[],
                                 max_pipelines=5,
                                 optimize_thresholds=True)

automl_auc.search(X_train, y_train)

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. 
Greater score is better.

Searching up to 5 pipelines. 
Allowed model families: xgboost, random_forest, catboost, linear_model



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

(1/5) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.500
(2/5) CatBoost Classifier w/ Simple Imputer    Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean AUC: 0.933
(3/5) XGBoost Classifier w/ Simple Imputer ... Elapsed:00:25
	Starting cross validation
	Finished cross validation - mean AUC: 0.724
(4/5) Random Forest Classifier w/ Simple Im... Elapsed:00:25
	Starting cross validation
	Finished cross validation - mean AUC: 0.708
(5/5) Logistic Regression Classifier w/ Sim... Elapsed:00:27
	Starting cross validation
	Finished cross validation - mean AUC: 0.702

Search finished after 00:28            
Best pipeline: CatBoost Classifier w/ Simple Imputer
Best pipeline AUC: 0.932842


like before, we can look at the rankings and pick the best pipeline

In [12]:
automl_auc.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,1,CatBoost Classifier w/ Simple Imputer,0.932842,False,{'Simple Imputer': {'impute_strategy': 'most_f...
1,2,XGBoost Classifier w/ Simple Imputer + One Hot...,0.723820,False,{'Simple Imputer': {'impute_strategy': 'most_f...
2,3,Random Forest Classifier w/ Simple Imputer + O...,0.708460,False,{'Simple Imputer': {'impute_strategy': 'most_f...
3,4,Logistic Regression Classifier w/ Simple Imput...,0.702148,False,{'Simple Imputer': {'impute_strategy': 'most_f...
4,0,Mode Baseline Binary Classification Pipeline,0.500000,False,{'Baseline Classifier': {'strategy': 'random_w...


In [13]:
best_pipeline_auc = automl_auc.best_pipeline

# train on the full training data
best_pipeline_auc.fit(X_train, y_train)

<evalml.pipelines.utils.make_pipeline.<locals>.GeneratedPipeline at 0x154986c10>

In [14]:
# get the auc and lead scoring score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout,  objectives=["auc", lead_scoring_objective])

OrderedDict([('AUC', 0.9427319431852523),
             ('Lead Scoring', 11.986242476354256)])

When we optimize for AUC, we can see that the AUC score from this pipeline is better than the AUC score from the pipeline optimized for lead scoring. However, the revenue per lead gained was only `$7` per lead when optimized for AUC and was `$45` when optimized for lead scoring. As a result, we would gain up to 6x the amount of revenue if we optimized for lead scoring.

This happens because optimizing for AUC does not take into account the user-specified true_positive (dollar amount to be gained with a successful lead) and false_positive (dollar amount to be lost with an unsuccessful lead) values. Thus, the best pipelines may produce the highest AUC but may not actually generate the most revenue through lead scoring.

This example highlights how performance in the real world can diverge greatly from machine learning metrics.